# qiskit_alt

This python package provides a thin wrapper around some features of Qiskit that have been (re-)implemented in Julia and provides a Python interface. The input and output of the Python interface are the same as the input and output to Python qiskit. At present, we have prepared two high level demonstrations

* Performing the Jordan-Wigner transform from a Fermionic operator to a Pauli operator.

* Computing the Fermionic operator from integrals computed by `pyscf`.

In both cases, we will see that the `qiskit_alt` implementation is much more performant.

We have also prepared some lower-level demonstrations of performance gains

* Converting an operator from the computational basis to the Pauli basis.

* Creating a `SparsePauliOp` from a list of strings

The Python package has been installed in a virtual environment created with `python -m venv ./env`. The Julia packages have been installed in a local environment in the standard way, via a spec in `Project.toml` file.

When we import the package `qiskit_alt`, the Julia environment is also activated.

In [1]:
%time import qiskit_alt  # The message below is from Julia

  Activating project at `~/myrepos/quantum_repos/qiskit_alt`


CPU times: user 1.65 s, sys: 1.6 s, total: 3.25 s
Wall time: 1.86 s


We assume that no one is familiar with Julia, much less with `pyjulia`, the package we use to call Julia from Python. So, we inject a bit of tutorial.

The default `Module` in Julia `Main` is available. You can think of it as a namespace. And, as always, objects from the `Module` `Base` have been imported into `Main`.

As an example of how `pyjulia` works, we create an `Array` of `Float64` zeros on the Julia side, and they are automatically copied to a numpy array when returned to Python.

In [2]:
qiskit_alt.Main.zeros(10)

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall ~/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

There are several ways to call Julia from Python and vice versa, and to specifiy features such as the copying vs. sharing semantics. We won't go into any of this in this demo.

## Electronic structure

Part of a workflow for, say, VQE involves using qiskit-nature to do the following:
* Get a description of a model Hamiltonian from the package `pyscf` by passing it a description of the geometry of a molecule.
* Convert that description of a Hamiltonian to a qiskit-native Fermionic operator.
* Convert the Fermionic operator to a qubit operator expressed in the Pauli basis.

The last step above may be done in several ways, one of which is known as the Jordan-Wigner transform. It is this step that we will benchmark here.

### qiskit-nature

First, we see how this is done in qiskit-nature. We need to specify the geometry of the molecule and the
[basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)). We choose `sto3g`, one of the smallest, simplest, basis sets.

In [3]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver

# Specify the geometry of the H_2 molecule
geometry = [['H', [0., 0., 0.]],
            ['H', [0., 0., 0.735]]]

basis = 'sto3g'

Then, we compute the fermionic Hamiltonian like this.

In [4]:
molecule = Molecule(geometry=geometry,
                     charge=0, multiplicity=1)
driver = ElectronicStructureMoleculeDriver(molecule, basis=basis, driver_type=ElectronicStructureDriverType.PYSCF)

from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()

fermionic_hamiltonian = second_q_op[0]

The Jordan-Wigner transform is performed like this.

In [5]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
nature_qubit_op = qubit_converter.convert(fermionic_hamiltonian)
nature_qubit_op.primitive

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.81054798+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

### qiskit_alt

The only high-level code in `qiskit_alt` was written to support this demo. So doing the JW-transform is less verbose.

In [6]:
fermi_op = qiskit_alt.fermionic_hamiltonian(geometry, basis)
pauli_op = qiskit_alt.jordan_wigner(fermi_op)
pauli_op.simplify() # The Julia Pauli operators use a different sorting convention; we sort again for comparison. 

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.09057899+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

Note that the constant term differs. The qiskit-nature version ignores the nuclear-repulsion term. I need to open an issue about whether and how to handle it.

### Benchmarking

Computing the Hamiltonian for a larger molecule or a larger basis set takes more time and produces a Hamiltonian with more factors and terms. Here we compare the performance of `qiskit_alt` and `qiskit-nature` on combinations of $\text{H}_2$ and $\text{H}_2\text{O}$ molecules for several basis sets.

First we benchmark qiskit-nature, and record the times in `nature_times`.

In [7]:
%run ../bench/jordan_wigner_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 8.05 ms
geometry=h2_geometry, basis='631g' 107.20 ms
geometry=h2_geometry, basis='631++g' 565.85 ms
geometry=h2o_geometry, basis='sto3g' 664.46 ms
geometry=h2o_geometry, basis='631g' 19270.12 ms


[8.052972005680203,
 107.19696339219809,
 565.853479411453,
 664.4582789391279,
 19270.118192769587]

Next we benchmark qiskit_alt, and record the times in `alt_times`.

In [8]:
%run ../bench/jordan_wigner_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 0.81 ms
geometry=h2_geometry, basis='631g' 4.68 ms
geometry=h2_geometry, basis='631++g' 24.82 ms
geometry=h2o_geometry, basis='sto3g' 21.95 ms
geometry=h2o_geometry, basis='631g' 713.85 ms


[0.8074820972979069,
 4.684781422838569,
 24.82047537341714,
 21.950607793405652,
 713.8539997860789]

We compare the relative performance.

In [9]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[9.972941855464066,
 22.881956214564667,
 22.797850198207126,
 30.27060959736809,
 26.994480942243477]

We see that
* qiskit_alt is at least ten times faster
* The relative performance increases as the problem in some sense gets larger.

In fact, another problem, not shown here, finishes in 18s with qiskit_alt and in 5730s in qiskit-nature.
In this case, `qiskit_alt` is 320 times faster than `qiskit-nature`. I don't have an idea about the origin of this scaling.

### Computing the Fermonic operator

Computing the Fermionic operator from the output of `pyscf` is also much more efficient in `qiskit_alt`.

We benchmark qiskit-nature computing the fermionic Hamiltonian

In [10]:
%run ../bench/fermionic_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 61.16 ms
geometry=h2_geometry, basis='631g' 102.76 ms
geometry=h2_geometry, basis='631++g' 305.58 ms
geometry=h2o_geometry, basis='sto3g' 460.60 ms
geometry=h2o_geometry, basis='631g' 26367.53 ms


[61.15992539562285,
 102.76125096715987,
 305.5791581980884,
 460.5999795719981,
 26367.526400834322]

We benchmark qiskit_alt computing the fermionic Hamiltonian.

In [11]:
%run ../bench/fermionic_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 60.64 ms
geometry=h2_geometry, basis='631g' 63.66 ms
geometry=h2_geometry, basis='631++g' 67.90 ms
geometry=h2o_geometry, basis='sto3g' 79.34 ms
geometry=h2o_geometry, basis='631g' 193.29 ms


[60.644444124773145,
 63.6613757815212,
 67.90349232032895,
 79.33510621078312,
 193.2857227511704]

We compare the relative performance.

In [12]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[1.008500057644013,
 1.6141852057961348,
 4.500197968560213,
 5.805752353166875,
 136.41735160531746]

We see again that, as the problem size increases, `qiskit_alt` is increasingly more performant.

## Discussion

The Julia implemenation consists of these packages

* [`QuantumOps.jl`](https://github.ibm.com/John-Lapeyre/QuantumOps.jl) implementing Fermionic and Pauli operators and calculations using them.

* [`ElectronicStructure.jl`](https://github.ibm.com/John-Lapeyre/ElectronicStructure.jl) provides an interface to electronic structure packages.

* [`ElectronicStructurePySCF.jl`](https://github.ibm.com/John-Lapeyre/ElectronicStructurePySCF.jl) provides an interface to `pyscf`

* [`QiskitQuantumInfo.jl`](https://github.ibm.com/John-Lapeyre/QiskitQuantumInfo.jl) provides data structures that mirror Python Qiskit data structures. These are used as intermedidate structures for converting from `QuantumOps` and `ElectronicStructure` to Python Qiskit. In the future these might be used directly for calculations.


The Python interface is a Python package `qiskit_alt`. This could contain a mixture of Julia and Python code. Or all the Julia code might be moved to the Julia packages.

### Implementation

In the examples above, the following happens.

* Julia code calls `pyscf` and stores the results in Julia data structures.

* These data are used to construct a Fermionic operator as a data structure defined in `QuantumOps`.

* The Jordan-Wigner transform, implemented in `QuantumOps` is used to compute a Pauli operator.

* The Pauli operator (as a structure in `QuantumOps`) is converted to a Qiskit-like operator defined in `QiskitQuantumInfo.jl`.

* The operator defined in `QiskitQuantumInfo.jl` is sent to Python and converted to numpy arrays, which are then used to construct native Qiskit types. The conversion to numpy arrays is provided by `pyjulia`.

### Complexity, dynamism

* It is worth noting that operators in `QuantumOps` are *not* highly optimized implementations. In fact, much of the code for the two types of operators is shared, they inherit from a parent class. There are other implementations of Pauli operators in Julia that are much more efficient for instance in [`QuantumClifford.jl`](https://github.com/Krastanov/QuantumClifford.jl).

* [Issue](https://github.ibm.com/John-Lapeyre/QuantumOps.jl/issues/6) for improving performance of Jordan-Wigner in `QuantumOps`.
    * Precompute one and two-body terms
    * Use @threads


# More demos

Here are some smaller scale demonstrations.

## Converting a matrix to the Pauli basis

Here we convert a matrix representing an operator in the computational basis to the Pauli basis.
In this case, `qiskit_alt` is much more performant than `qiskit.quantum_info`.
This is how it is done in `QuantumOps`.

In [13]:
from qiskit_alt import QuantumOps, PauliSum_to_SparsePauliOp
import numpy as np
m = np.random.rand(2**3, 2**3) # 3-qubit operator
pauli_sum = QuantumOps.PauliSum(m) # This is a wrapped Julia object
PauliSum_to_SparsePauliOp(pauli_sum) # Convert to qiskit.quantum_info.SparsePauliOp

SparsePauliOp(['III', 'XII', 'YII', 'ZII', 'IXI', 'XXI', 'YXI', 'ZXI', 'IYI', 'XYI', 'YYI', 'ZYI', 'IZI', 'XZI', 'YZI', 'ZZI', 'IIX', 'XIX', 'YIX', 'ZIX', 'IXX', 'XXX', 'YXX', 'ZXX', 'IYX', 'XYX', 'YYX', 'ZYX', 'IZX', 'XZX', 'YZX', 'ZZX', 'IIY', 'XIY', 'YIY', 'ZIY', 'IXY', 'XXY', 'YXY', 'ZXY', 'IYY', 'XYY', 'YYY', 'ZYY', 'IZY', 'XZY', 'YZY', 'ZZY', 'IIZ', 'XIZ', 'YIZ', 'ZIZ', 'IXZ', 'XXZ', 'YXZ', 'ZXZ', 'IYZ', 'XYZ', 'YYZ', 'ZYZ', 'IZZ', 'XZZ', 'YZZ', 'ZZZ'],
              coeffs=[ 0.50984036+0.00000000e+00j,  0.37520081+0.00000000e+00j,
  0.        -1.25691042e-01j,  0.15333533+0.00000000e+00j,
  0.43236486+0.00000000e+00j,  0.4673399 +0.00000000e+00j,
  0.        -9.29061785e-02j,  0.10976694+0.00000000e+00j,
  0.        +1.24000327e-02j,  0.        +2.26919435e-01j,
  0.18554457+0.00000000e+00j,  0.        -1.06701287e-01j,
 -0.11396592+0.00000000e+00j,  0.02640026+0.00000000e+00j,
  0.        +1.84139363e-04j,  0.00822458+0.00000000e+00j,
  0.4103655 +0.00000000e+00j,  0.46898728+0

Note that the `numpy` matrix was automatically converted to a Julia `Matrix`.

We run benchmarks of conversion of matrices to the Pauli basis.

In [14]:
%run ../bench/from_matrix_quantum_info.py

nqubits=2, 1.18 ms
nqubits=3, 4.85 ms
nqubits=4, 25.18 ms
nqubits=5, 137.54 ms
nqubits=6, 1009.72 ms
nqubits=7, 8799.43 ms
nqubits=8, 108867.59 ms


In [15]:
%run ../bench/from_matrix_alt.py

nqubits=2, 0.75 ms
nqubits=3, 0.84 ms
nqubits=4, 3.49 ms
nqubits=5, 5.23 ms
nqubits=6, 22.39 ms
nqubits=7, 110.32 ms
nqubits=8, 656.34 ms


Here are the ratios of the times for `qiskit.quantum_info` to those for `qiskit_alt`.

In [16]:
[t_pyqk / t_qk_alt for t_pyqk, t_qk_alt in zip(pyqk_times, qk_alt_times)]

[1.5658879614192156,
 5.739860256223572,
 7.2102312566595685,
 26.317532936650075,
 45.089134443160745,
 79.76375050261795,
 165.87083460712748]

Again, the performance gain increases with the problem size.

## Creating a `SparsePauliOp` from a list of strings


Here, we create a `SparsePauliOp` from a list of `n` strings, each with `k` single-Pauli factors, and simplify the result.

First, using `qiskit.quantum_info`

In [17]:
%run ../bench/pauli_from_list_qinfo.py

k=10, n=10, 0.3379025496542454 ms
k=10, n=100, 1.5436640474945307 ms
k=10, n=1000, 13.85800545103848 ms
k=10, n=5000, 67.62067864183336 ms
k=10, n=10000, 137.34887726604939 ms
k=10, n=100000, 1396.8325019814074 ms
k=100, n=10, 0.3659903537482023 ms
k=100, n=100, 1.6624061157926917 ms
k=100, n=1000, 14.924054243601859 ms
k=100, n=5000, 73.60771629028022 ms
k=100, n=10000, 151.67049085721374 ms
k=100, n=100000, 1534.970625769347 ms


Now, using `qiskit_alt`

In [18]:
%run ../bench/pauli_from_list_alt.py

k=10, n=10, 0.698221568018198 ms
k=10, n=100, 0.7741700625047088 ms
k=10, n=1000, 1.9150953041389585 ms
k=10, n=5000, 9.954712656326592 ms
k=10, n=10000, 18.018387840129435 ms
k=10, n=100000, 203.53047845419496 ms
k=100, n=10, 0.6988641107454896 ms
k=100, n=100, 0.8893186692148447 ms
k=100, n=1000, 6.8468364188447595 ms
k=100, n=5000, 13.74192121438682 ms
k=100, n=10000, 30.91609589755535 ms
k=100, n=100000, 357.74553280789405 ms


The results were written to lists `quantum_info_times` and `qkalt_times`. We compare the performance:

In [19]:
[x / y for x,y in zip(quantum_info_times, qkalt_times)]

[0.4839474532609089,
 1.9939598833107055,
 7.236196246258953,
 6.792830790435513,
 7.622706231250861,
 6.863013896445824,
 0.5236931588285375,
 1.8693030668752124,
 2.179700715870168,
 5.356435620749895,
 4.9058746408276885,
 4.290677269179519]

We see that the performance improvement in `qiskit_alt` is significant, but does not increase with the number of terms `n`. Further benchmarks show that the time required to convert the strings from Python to Julia takes all the time.

We see this in the following.

In [20]:
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


Generate `1000` ten-qubit Pauli strings.

In [21]:
%julia using Random: randstring
%julia pauli_strings = [randstring("IXYZ", 10) for _ in 1:1000]
None;

Benchmark converting these to a `QuantumOps.PauliSum`. Note that as the sums are always sorted.

In [22]:
%julia using BenchmarkTools: @btime
%julia @btime QuantumOps.PauliSum($pauli_strings)
None;

  352.993 μs (1012 allocations: 134.17 KiB)


Check that we are actually getting the right result.

In [23]:
%julia pauli_sum = QuantumOps.PauliSum(pauli_strings);
%julia println(length(pauli_sum))
%julia println(pauli_sum[1])

1000
10-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Complex{Int64}}:
IIIIIIZIYZ * (1 + 0im)


In [24]:
6.9 * 2.29 / .343  # Ratio of time to construct PauliSum via qiskit_alt to time in pure Julia

46.06705539358601

So the pure Julia code is `46` times faster than the qiskit.quantum_info.
**But, the `qiskit.quantum_info` is also extracting a possible phase !**